In [1]:
%load_ext autoreload
%autoreload 2

## Note to visualize and compare the estimated lifetimes for DBA turbines from the RULe method's fatigue tables vs. structural reports

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from utils.setup_custom_logger import setup_custom_logger
import os 
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [30 / 2.54, 20 / 2.54]
pd.set_option('display.width', 1000) # Print columns wider
pd.set_option('display.max_rows', 500) # Print more rows

In [3]:
logger = setup_custom_logger('lifetime_summary')    
res_base_dir = fr'{os.getcwd()}\output\all_turbines'
df = pd.read_excel(os.path.join(res_base_dir, 'all_lifetimes_from_fatigue_tables.xlsx'), index_col=None)
df.rename(columns = {'min_lifetime': 'lifetime'}, inplace = True)


In [4]:
print(df.groupby('cluster').count())
print(df.groupby('cluster')['lifetime'].mean())
print(df.groupby('cluster')['lifetime'].min())
print(df.groupby('cluster')['lifetime'].max())

         turbine_name  lifetime
cluster                        
JLN                26        26
JLO                47        47
JLP                22        22
cluster
JLN    34.400958
JLO    33.765762
JLP    47.470156
Name: lifetime, dtype: float64
cluster
JLN    34.400958
JLO    31.971665
JLP    44.466338
Name: lifetime, dtype: float64
cluster
JLN    34.400958
JLO    39.629254
JLP    56.752639
Name: lifetime, dtype: float64


In [5]:
fig1_w = go.FigureWidget()
fig1_w.layout = go.Layout(xaxis=dict(range=[int(df['lifetime'].min() * 0.8), int(df['lifetime'].max() * 1.1)]), title='Fatigue lifetime distribution of Dogger Bank A turbines according to RULe fatigue lookup tables', xaxis_title='Years', yaxis_title='Counts')
fig1_w.add_trace(go.Histogram(x = df[df['cluster'] == 'JLO']['lifetime'], opacity=0.8, name='Intermediate'))
fig1_w.add_trace(go.Histogram(x = df[df['cluster'] == 'JLN']['lifetime'], opacity=0.8, name='Deep'))
fig1_w.add_trace(go.Histogram(x = df[df['cluster'] == 'JLP']['lifetime'], opacity=0.8, name='Shallow'))
fig1_w.update_layout(barmode='stack')

FigureWidget({
    'data': [{'name': 'Intermediate',
              'opacity': 0.8,
              'type': 'histogram',
              'uid': '737c2664-55a2-4437-ae34-94989ad785cc',
              'x': array([33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 31.9716647 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 , 33.6919512 ,
                          33.6919512 , 32.94403893, 33.6919512 , 33.6919512 , 33.6919512 ,
  

In [6]:
df_report = pd.read_excel( os.path.join(res_base_dir, 'structural_report_inplace_lifetimes.xlsx'), index_col=None) # reported worst in place util

In [7]:
df_report_Dd_tot = pd.read_excel( os.path.join(res_base_dir, 'structural_report_Dd_tot_lifetimes.xlsx'), index_col=None) # reported worst Dd tot util

In [21]:
new_df = pd.DataFrame()

for name in df['turbine_name']:
    reported_elevation = df_report[df_report['turbine_name'] == name].copy()
    reported_elevation['lookup_table_lifetime'] = float(df[df['turbine_name'] == name]['lifetime'])
    new_df = pd.concat( [new_df, reported_elevation] )
    
new_df.columns

Index(['turbine_name', 'cluster', 'elevation', 'in_out', 'description', 'in_place_utilization', 'lifetime', 'lookup_table_lifetime'], dtype='object')

In [48]:
def identify_subseabed_points(descr):
    if 'CW' in descr:
        n = int(descr.split('CW-')[1])
        if n >= 10:
            return f'{descr} (Sub Seabed)'    
    return descr       

In [49]:
def plot_lifetime_histogram_based_on_point(lifetime_df, lifetime_id = 'lifetime', plot_title = 'design utilization'):

    fw = go.FigureWidget()
    fw.layout = go.Layout(xaxis=dict(range=[25,60]), title=f'Fatigue lifetime distribution of Dogger Bank A turbines according to {plot_title}', xaxis_title='Years', yaxis_title='No. of turbines')

    for descr in lifetime_df['description'].unique():
        these_df = lifetime_df[lifetime_df['description'] == descr]
        for cluster in these_df['cluster'].unique():
            fw.add_trace(go.Histogram(x = these_df[these_df['cluster'] == cluster][lifetime_id], opacity=0.8, name= f"{cluster} {identify_subseabed_points(descr)}"))
        
    fw.update_layout(barmode='stack')
    return fw

plot_lifetime_histogram_based_on_point(new_df, lifetime_id='lifetime')
    

FigureWidget({
    'data': [{'name': 'JLO Top Boat landing Support',
              'opacity': 0.8,
              'type': 'histogram',
              'uid': 'e6e588ac-b566-4ebb-a989-db74b1118971',
              'x': array([33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546, 33.89236546,
                          33.89236546, 33.89236546, 33.89236546, 33.89236546]

In [50]:
plot_lifetime_histogram_based_on_point(new_df, lifetime_id='lookup_table_lifetime', plot_title='RULe lookup tables')

FigureWidget({
    'data': [{'name': 'JLO Top Boat landing Support',
              'opacity': 0.8,
              'type': 'histogram',
              'uid': '0990a5f0-b68e-496c-9aa4-f12854344d9c',
              'x': array([33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512, 33.6919512,
                          33.6919512, 33.6919512])},
             {'name': 'JLN Top Boat landing Support',
            

In [75]:
deviation_df = new_df[(new_df['lookup_table_lifetime'] - new_df['lifetime']) > 2.0 ]
deviation_df

,turbine_name,cluster,elevation,in_out,description,in_place_utilization,lifetime,lookup_table_lifetime
71,DA_P57_DE,JLO,3.92,I,CW-2,92.0,29.434783,39.629254


In [120]:
turbine_to_correct = 'DA_P57_DE'
increase_damage_by_factor = deviation_df['lookup_table_lifetime'] / deviation_df['lifetime']
increase_damage_by_factor

71    1.346341
dtype: float64

In [121]:

file_format = '.json' # xlsx or json
file_name_key = 'fatigue_damage' # lookup_table or fatigue_damage
file_loc = "blob" # blob or all_turbines

res_base_dir = os.path.join( os.getcwd(), "output", file_loc)
paths_to_lookup_tables = [os.path.join(path, name) for path, subdirs, files in os.walk(res_base_dir) 
                            for name in files if ((file_name_key in name) and (file_format) in name)]

path_to_lookup_table = [p for p in paths_to_lookup_tables if turbine_to_correct in p][0]
path_to_lookup_table

'c:\\Appl\\TDI\\fatigue-calculations\\baseline_methods\\output\\blob\\fatigue_damage_all_dlcs_scaled_DBA_DA_P57_DE.json'

In [127]:
lookup_table_df = pd.read_json(path_to_lookup_table, dtype=False)

In [128]:
import numpy as np

In [129]:
lookup_table_df.loc[:, lookup_table_df.columns.str.contains('sector')] = lookup_table_df.loc[:, lookup_table_df.columns.str.contains('sector')].astype(np.float64)
lookup_table_df

,index,DLC,Nacelle angle [deg],Wind Speed [m/s],Wind Direction [deg],Wave direction [deg],Hs windsea [m],Tp windsea [s],Hs swell [m],Tp swell [s],...,fatigue_damage_sector_14,fatigue_damage_sector_15,fatigue_damage_sector_16,fatigue_damage_sector_17,fatigue_damage_sector_18,fatigue_damage_sector_19,fatigue_damage_sector_20,fatigue_damage_sector_21,fatigue_damage_sector_22,fatigue_damage_sector_23
0,0,DLC12,0,4.0,0,0,0.26,3.11,1.12,8.43,...,8.206727e-09,6.465040e-09,3.894931e-09,1.769148e-09,6.610470e-10,2.526600e-10,1.884470e-10,4.635220e-10,1.354751e-09,3.272280e-09
1,1,DLC12,8,4.0,0,0,0.26,3.11,1.12,8.43,...,1.954589e-08,1.694442e-08,1.116139e-08,5.642754e-09,2.258157e-09,8.912860e-10,4.727750e-10,5.301290e-10,1.687145e-09,5.129249e-09
2,2,DLC12,-8,4.0,0,0,0.26,3.11,1.12,8.43,...,1.196065e-08,7.974133e-09,3.669437e-09,1.036025e-09,1.618370e-10,2.973800e-11,7.312200e-11,4.688450e-10,2.012758e-09,5.514883e-09
3,3,DLC12,0,4.0,0,30,0.26,3.11,1.12,8.43,...,6.234105e-09,5.716056e-09,3.889677e-09,1.953589e-09,7.239700e-10,2.216810e-10,1.210270e-10,2.534070e-10,6.847320e-10,1.744164e-09
4,4,DLC12,8,4.0,0,30,0.26,3.11,1.12,8.43,...,1.552829e-08,1.436046e-08,1.018151e-08,5.577560e-09,2.422841e-09,8.367630e-10,3.566720e-10,5.350400e-10,1.582810e-09,4.130617e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10705,3754,DLC64b,322,30.0,330,330,8.22,13.49,1.13,8.45,...,2.110292e-05,2.052613e-05,1.622739e-05,1.011650e-05,4.420952e-06,1.348078e-06,5.216686e-07,6.254355e-07,1.897731e-06,5.463768e-06
10706,3755,DLC64b,338,30.0,330,330,8.22,13.49,1.13,8.45,...,6.199035e-05,4.406796e-05,2.354061e-05,7.965492e-06,1.429341e-06,4.662778e-07,2.292241e-06,1.178508e-05,2.915119e-05,4.944224e-05
10707,3756,DLC64b,330,31.0,330,330,9.05,14.31,0.71,7.84,...,2.114280e-05,1.860081e-05,1.323917e-05,7.010476e-06,2.538721e-06,9.984347e-07,7.794698e-07,1.250705e-06,3.476459e-06,8.228020e-06
10708,3757,DLC64b,322,31.0,330,330,9.05,14.31,0.71,7.84,...,3.454079e-05,3.482720e-05,2.887625e-05,1.907813e-05,9.165572e-06,2.722323e-06,4.870856e-07,3.790098e-07,2.164039e-06,8.131828e-06


In [131]:
lookup_table_df.loc[:, lookup_table_df.columns.str.contains('sector')] = lookup_table_df.loc[:, lookup_table_df.columns.str.contains('sector')].apply(lambda row: row * increase_damage_by_factor, axis=1)
lookup_table_df
# TODO this just makes all values Nan

,index,DLC,Nacelle angle [deg],Wind Speed [m/s],Wind Direction [deg],Wave direction [deg],Hs windsea [m],Tp windsea [s],Hs swell [m],Tp swell [s],...,fatigue_damage_sector_14,fatigue_damage_sector_15,fatigue_damage_sector_16,fatigue_damage_sector_17,fatigue_damage_sector_18,fatigue_damage_sector_19,fatigue_damage_sector_20,fatigue_damage_sector_21,fatigue_damage_sector_22,fatigue_damage_sector_23
0,0,DLC12,0,4.0,0,0,0.26,3.11,1.12,8.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,DLC12,8,4.0,0,0,0.26,3.11,1.12,8.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,DLC12,-8,4.0,0,0,0.26,3.11,1.12,8.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,DLC12,0,4.0,0,30,0.26,3.11,1.12,8.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,DLC12,8,4.0,0,30,0.26,3.11,1.12,8.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10705,3754,DLC64b,322,30.0,330,330,8.22,13.49,1.13,8.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10706,3755,DLC64b,338,30.0,330,330,8.22,13.49,1.13,8.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10707,3756,DLC64b,330,31.0,330,330,9.05,14.31,0.71,7.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10708,3757,DLC64b,322,31.0,330,330,9.05,14.31,0.71,7.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

from lifetime_calculation_from_lookup import calculate_lifetime_from_fatigue_lookup_table
print( calculate_lifetime_from_fatigue_lookup_table(lookup_table_df) )